In [ ]:
from nerus.log import log_progress
from nerus.dump import load_raw
from nerus.path import join_path
from nerus.const import (
    DUMPS_DIR, RAW, JSONL_GZ,
    FACTRU, NE5,
    PER, LOC, ORG
)
from nerus.eval import eval_markups


path = join_path(DUMPS_DIR, RAW, FACTRU + JSONL_GZ)
records = log_progress(load_raw(path), prefix=FACTRU)
factru = [_ for _ in records if int(_.source.id) >= 3000]  # testset

path = join_path(DUMPS_DIR, RAW, NE5 + JSONL_GZ)
records = log_progress(load_raw(path), prefix=NE5)
ne5 = list(records)

# F1

In [ ]:
scores = {}
for label, dataset in [(FACTRU, factru), (NE5, ne5)]:
    etalon = [_.source.adapted for _ in log_progress(dataset, prefix=label)]
    guess = [_.adapted.mixed for _ in log_progress(dataset, prefix=label)]
    scores[label] = eval_markups(log_progress(guess, prefix=label), etalon)

In [ ]:
import pandas as pd


data = []
for source in [FACTRU, NE5]:
    for type in [PER, LOC, ORG]:
        f1 = scores[source][type].f1
        data.append([source, type, f1])

table = pd.DataFrame(data, columns=['source', 'type', 'f1'])
table = table.pivot('source', 'type', 'f1')
table = table.reindex(index=[FACTRU, NE5], columns=[PER, LOC, ORG])
display(table)
print(table.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>type</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
    </tr>
    <tr>
      <th>source</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>factru</th>
      <td>0.966348</td>
      <td>0.910089</td>
      <td>0.792040</td>
    </tr>
    <tr>
      <th>ne5</th>
      <td>0.986165</td>
      <td>0.930071</td>
      <td>0.849115</td>
    </tr>
  </tbody>
</table>

# Error examples

In [ ]:
from random import seed, sample

from ipymarkup import show_ascii_markup as show_markup

from nerus.utils import iter_sents


seed(1)
records = sample(ne5, 10)
sents = list(iter_sents(records))
for sent in sents:
    etalon = sent.source.adapted
    guess = sent.adapted.mixed
    if etalon == guess:
        continue
    print('etalon')
    show_markup(etalon.text, etalon.spans)
    print('guess')
    show_markup(guess.text, guess.spans)
    print('---')